In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/Colab Notebooks/evaluation/project14.h5"

'/content/drive/My Drive/Colab Notebooks/evaluation/project14.h5'


In [0]:
import numpy as np 
import pandas as pd 
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [0]:
def decode(text):
    text = text.encode('ascii','ignore')
    return text

In [0]:
data2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/evaluation/train.csv')
data=data2.copy()
pd.set_option('display.max_colwidth',-1)
data.head()

,Scene,Sentiment
0,"{Monica} How about, youre moving!! {Rachel} Look! This is ridiculous. We should be packing you!! {Phoebe} Hey, how are you guys doing? {Rachel} Great! Monicas moving! {Monica} I am not! {Rachel} Oh really?! Then how come all your stuff is in this box?! {Phoebe} Okay, you guys. You guys I think I know whats going on here. Okay, you guys {Monica} No Phoebe I am mad! {Phoebe} Well, deep-deep-deep down! {Rachel} Yeah, Im just mad! {Phoebe} Then keep running.",NEGATIVE
1,{Phoebe} Or we could just follow your clever jokes  any ideas?,NEUTRAL
2,"{Joey} Hey Rach! Hey, you mind if I read my comic books in here? {Rachel} Sure! Why? {Joey} Oh well, Chandler and Monica are over there and it's kinda hard to concentrate. {Rachel} What?! {Rachel} She just called and said that she was gonna be working late! {Rachel} She keeps lying to me! {Rachel} That's it! {Rachel} I'm just gonna go over there and confront them right now!",NEGATIVE
3,"{Joey} You forget how many great songs Heart had. {Chandler} Yeah. {Ross} You know, Barracuda was the first song I learned to play on the keyboard. {Chandler} So, you heard it, you repeated it, so that must mean you wrote it. {Joey} Oh, you guys, with this joke. I gotta say, I know I cracked up, but Im not even sure I got it. {Ross} What, you didnt get it? The doctor is a monkey. {Chandler} And monkeys cant write out prescriptions. {Chandler} You are not allowed to laugh at my joke. {Ross} Your joke? Well, I think the Hef would disagree, which is why he sent me a check for one hundred ah-dollars.",NEUTRAL
4,"{Chandler} Hey Joe! You wanna shoot some hoops? {Joey} Oh no, I cant go. Im practicing; I got an audition to be the host of a new game show.",POSTIVE


In [0]:
import tensorflow as tf
subsetDataFrame = data[data['Sentiment'] == 'MIXED']
m=subsetDataFrame.copy()
sub2=data[data['Sentiment'] == 'POSTIVE']
sub2=sub2.iloc[0:115]
print(len(subsetDataFrame))
data=data.append(m,ignore_index=True)
data=data.append(m,ignore_index=True)
data=data.append(m,ignore_index=True)
data=data.append(m,ignore_index=True)
data=data.append(sub2,ignore_index=True)
print(len(data))
label = data['Sentiment']
pos=0
neg=0
mixed=0
neutral=0
for i in label:
  if i=='POSTIVE':
    pos=pos+1
  elif i=='NEGATIVE':
    neg=neg+1
  elif i=='MIXED':
    mixed=mixed+1
  else:
    neutral=neutral+1
print(pos)
print(neg)
print(mixed)
print(neutral)
tx=data['Scene'].values


y_label=[]
class_names = {'POSTIVE' :0, 'NEUTRAL':1, 'NEGATIVE':2, 'MIXED':3}
for x in label:
  y_label.append(class_names[str(x)])

y=tf.keras.utils.to_categorical(y_label,num_classes=4)

#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)
#print(X_train.shape,y_train.shape)
#print(X_test.shape,y_test.shape)

60
1355
347
367
300
341


In [0]:
#tx=data['Scene'].values
print(tx[1])
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
  
stop_words = set(stopwords.words('english')) 
tx2=[]
for x in tx:
  word_tokens = word_tokenize(x) 
  
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  filtered_sntence=[]
  string=""
  
  for w in word_tokens: 
      if w not in stop_words: 
          filtered_sentence.append(w)
  #data['Text'] = np.vectorize(decode)(data['Text'])        
  tx2.append(filtered_sentence)
  
print(tx2[0])
print(len(tx2))
#print(word_tokens) 
#print(filtered_sentence) 

{Phoebe} Or we could just follow your clever jokes  any ideas? 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
['{', 'Monica', '}', 'How', ',', 'you\x92re', 'moving', '!', '!', '{', 'Rachel', '}', 'Look', '!', 'This', 'ridiculous', '.', 'We', 'packing', '!', '!', '{', 'Phoebe', '}', 'Hey', ',', 'guys', '?', '{', 'Rachel', '}', 'Great', '!', 'Monica\x92s', 'moving', '!', '{', 'Monica', '}', 'I', '!', '{', 'Rachel', '}', 'Oh', 'really', '?', '!', 'Then', 'come', 'stuff', 'box', '?', '!', '{', 'Phoebe', '}', 'Okay', ',', 'guys', '.', 'You', 'guys', 'I', 'think', 'I', 'know', 'what\x92s', 'going', '.', 'Okay', ',', 'guys', '{', 'Monica', '}', 'No', 'Phoebe', 'I', 'mad', '!', '{', 'Phoebe', '}', 'Well', ',', 'deep-dee

In [0]:
from sklearn.preprocessing import Normalizer

num_words = 6000
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\nÂ',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(tx2)
X = tokenizer.texts_to_sequences(tx2)
print(len(X))

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 150
X =pad_sequences(X, maxlen=max_length_of_text)
print(len(X))
print(X[250])


1355
Found 5792 unique tokens.
1355
[   8    2   91    4   43  170   33    1   97  402    2  160   77  645
    3    6  452  109 2155 3875  391  115 3876    7    1    8    2  348
    7    5    1   97  402    2   25    3   15  489  257    4    1   97
  402    2    6  144  180  175 3877  364  195   19    6   27  151  545
   62    4    1   97  402    2   33  343    4    1   97  402    2 3878
   79    7    1    8    2   33    6 1998    4    1   97  402    2  419
 3879   27    4   21   53 2748 2749    4    1    8    2   21   53   42
  139    4    1   97  402    2  343    3  312   15   79  115   67    4
   50 1408  322    3   15  121  695  383    4    1    8    2   14  111
    4  133   63   42  896 1529    4    1   97  402    2  380  295    3
  295   42  896   15 1529    4   26   15  370    4]


In [0]:
embeddings_index = {}
f = open('/content/drive/My Drive/Colab Notebooks/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [0]:
EMBEDDING_DIM = 50
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_length_of_text,
                            trainable=False)

In [0]:
from keras.layers import Dense, Embedding, LSTM, Bidirectional
from keras.layers import Dense
from keras.layers import Dropout
lstm_out =128
batch_size =16
inputs2 = Input((max_length_of_text, ))
x2 = embedding_layer(inputs2)
#x2=Bidirectional(LSTM(lstm_out = 128, dropout= 0.3, recurrent_dropout= 0.2))(x2)
x2= LSTM(128, dropout=0.3, recurrent_dropout=0.25, return_sequences=False)(x2)
x2=Dense(64,activation='relu')(x2)
x2=Dropout(0.3)(x2)
x2=Dense(4,activation='softmax')(x2)
model = Model(inputs2, x2)
print(model.summary())

In [0]:
!pip install keras

In [0]:
import keras
print(keras.__version__)

In [0]:
from nltk.corpus import stopwords 

In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [0]:
filepath=r"/content/drive/My Drive/Colab Notebooks/evaluation/ev_trial20.h5"

import keras
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,save_weights_only=False, mode='max')

model.fit(X, y,validation_split=0.1, batch_size = batch_size, shuffle=True,epochs = 20,callbacks=[checkpoint])
model.save(r"/content/drive/My Drive/Colab Notebooks/evaluation/project20.h5")
with open(r"/content/drive/My Drive/Colab Notebooks/evaluation/tokenizer.pickle", 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
from keras.models import load_model
model=load_model(r"/content/drive/My Drive/Colab Notebooks/evaluation/.h5")
score,acc = model.evaluate(X, y, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

In [0]:
new3=load_model(r"/content/drive/My Drive/Colab Notebooks/evaluation/project13.h5")
new3.summary()
new3.fit(X, y,validation_split=0.075, batch_size = batch_size, shuffle=True,epochs = 10,callbacks=[checkpoint])
new3.save(r"/content/drive/My Drive/Colab Notebooks/evaluation/project13.h5")

In [0]:
new3=load_model(r"/content/drive/My Drive/Colab Notebooks/evaluation/project13.h5")
new3.summary()
new3.fit(X, y,validation_split=0.1, batch_size = batch_size, shuffle=True,epochs = 12,callbacks=[checkpoint])
new3.save(r"/content/drive/My Drive/Colab Notebooks/evaluation/project13.h5")

In [0]:
new3=load_model(r"/content/drive/My Drive/Colab Notebooks/evaluation/project13.h5")
new3.summary()
new3.fit(X, y,validation_split=0.1, batch_size = batch_size, shuffle=True,epochs = 5,callbacks=[checkpoint])
new3.save(r"/content/drive/My Drive/Colab Notebooks/evaluation/project13.h5")

In [0]:
new3=load_model(r"/content/drive/My Drive/Colab Notebooks/evaluation/project13.h5")
new3.summary()
new3.fit(X, y,validation_split=0.1, batch_size = batch_size, shuffle=True,epochs = 5,callbacks=[checkpoint])
new3.save(r"/content/drive/My Drive/Colab Notebooks/evaluation/project13.h5")

In [0]:
new3=load_model(r"/content/drive/My Drive/Colab Notebooks/evaluation/project13.h5")
new3.summary()
new3.fit(X, y,validation_split=0.1, batch_size = batch_size, shuffle=True,epochs = 5,callbacks=[checkpoint])
new3.save(r"/content/drive/My Drive/Colab Notebooks/evaluation/project14.h5")

In [0]:
new3=load_model(r"/content/drive/My Drive/Colab Notebooks/evaluation/project14.h5")
new3.summary()
new3.fit(X, y,validation_split=0.1, batch_size = batch_size, shuffle=True,epochs = 5,callbacks=[checkpoint])
new3.save(r"/content/drive/My Drive/Colab Notebooks/evaluation/project15.h5")

In [0]:
data2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/evaluation/test.csv')
pd.set_option('display.max_colwidth',-1)
data2.head()
tx2=data2['Scene'].values
y2=data['Sentiment'].values
print(tx2[1])
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
  
stop_words = set(stopwords.words('english')) 
tx3=[]
for x in tx2:
  word_tokens = word_tokenize(x) 
  
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
  filtered_sentence = [] 
  string=""
  for w in word_tokens: 
      if w not in stop_words: 
          filtered_sentence.append(w) 
  tx3.append(filtered_sentence)
print(tx3[2])
print(len(tx3))
#print(word_tokens) 
#print(filtered_sentence)


In [0]:
from sklearn.preprocessing import Normalizer

num_words = 8000
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\nÂ',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(tx3)
X_test = tokenizer.texts_to_sequences(tx3)
print(len(X_test))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 150
X_test =pad_sequences(X_test, maxlen=max_length_of_text)
print(len(X_test))

In [0]:
from keras.models import load_model
model=load_model(r"/content/drive/My Drive/Colab Notebooks/evaluation/ev_trial13.h5")
score,acc = model.evaluate(X_test, y, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

In [0]:
from keras.models import load_model
new=load_model("/content/drive/My Drive/Colab Notebooks/evaluation/project2.h5")
new.summary()
array=[[]]
label=[]
#print(model.predict(X_test,batch_size = batch_size))
arr=new.predict(X_test,batch_size = batch_size)
i=0
dictionary = {'0' :'POSTIVE', '1':'NEUTRAL', '2':'NEGATIVE', '3':'MIXED'}
while i<len(X_test):
  g=np.argmax(arr[i])
  label.append(dictionary[str(g)])
  i=i+1
print(len(label))  
df = pd.DataFrame({"Index":[i+1 for i in range(len(label))],"Sentiment":label})
df.to_csv('/content/drive/My Drive/Colab Notebooks/evaluation/sol130.csv',index=False)


ValueError: ignored